Se recolecta las funciones necesarias para el proyecto.

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews_recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [31]:
# Archivo primera y segunda funcion.
"""Para esta funcion necesitamos:
item_id: de los archivos Australian_users_items_normalizada y output_steam_games_limpio
genres: del archivo output_steam_games_limpio
playtime_forever: del archivo Australian_users_items_normalizada
user_id: del archivo items"""

import pandas as pd

# Cargar los archivos CSV en DataFrames
df_items = pd.read_csv('CSV/Australian_users_items_normalizada.csv')  
df_games = pd.read_csv('CSV/output_steam_games_limpio.csv')  

# Seleccionar las columnas que deseas combinar
columns_items = df_items[['item_id', 'playtime_forever','user_id']]
columns_games = df_games[['item_id', 'genres','release_year']]

# Combinar las columnas en un nuevo DataFrame
df_function_1_2 = pd.merge(columns_items, columns_games, on='item_id', how='inner')

# Guardar el nuevo DataFrame en un archivo CSV
df_function_1_2.to_csv('Datasets/function_1_2.csv', index=False)

In [50]:
# Comprobamos el archivo creado.
import pandas as pd
df_first = pd.read_csv('Datasets/function_1_2.csv')
df_first.head()

,item_id,playtime_forever,user_id,genres,release_year
0,10,6,76561197970982479,action,2000
1,10,93,doctr,action,2000
2,10,108,corrupted_soul,action,2000
3,10,328,WeiEDKrSat,action,2000
4,10,6275,death-hunter,action,2000


In [98]:
# primera funcion.

import pandas as pd

df_function_1 = pd.read_csv('Datasets/function_1_2.csv')

def PlayTimeGenre(genre:str):
    genre = genre.lower()

    df_function_1['playtime_forever'] = df_function_1['playtime_forever'] / 60

    df_filter = df_function_1[df_function_1['genres'] == genre]

    grouped = df_filter.groupby(['release_year'])['playtime_forever'].sum().reset_index()

    max_played_year = grouped.loc[grouped['playtime_forever'].idxmax()]['release_year']

    result = {"Año de lanzamiento con más horas jugadas para Género " + genre: int(max_played_year)}
    return result
print(PlayTimeGenre('rpg'))

{'Año de lanzamiento con más horas jugadas para Género rpg': 2011}


In [100]:
# segunda funcion.

import pandas as pd

df_function_2 = pd.read_csv('Datasets/function_1_2.csv')

def UserForGenre(genre:str):
    df_filter = df_function_2[df_function_2['genres'].str.lower() == genre.lower()]

    if df_filter.empty:
        return {"Mensaje": f"No se encontraron datos para el género {genre}"}

    # Agrupar por usuario y año
    grouped = df_filter.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index()
    grouped['playtime_forever'] = grouped['playtime_forever'] / 60  # Convertir minutos a horas

    # El usuario + horas jugadas por género
    max_user = grouped.groupby('user_id')['playtime_forever'].sum().idxmax()
    max_user_hours = grouped[grouped['user_id'] == max_user]

    # Obtener la lista de acumulación de horas jugadas por año
    hours_per_year = max_user_hours[['release_year', 'playtime_forever']]
    hours_list = [{"Año": int(year), "Horas": int(hours)} for year, hours in zip(hours_per_year['release_year'], hours_per_year['playtime_forever'])]

    result = {"Usuario con más horas jugadas para Género " + genre: max_user, "Horas jugadas": hours_list}
    return result
print(UserForGenre('action'))

{'Usuario con más horas jugadas para Género action': 'Cow666', 'Horas jugadas': [{'Año': 1994, 'Horas': 92}, {'Año': 1995, 'Horas': 0}, {'Año': 1997, 'Horas': 94}, {'Año': 2002, 'Horas': 1}, {'Año': 2003, 'Horas': 584}, {'Año': 2005, 'Horas': 3}, {'Año': 2008, 'Horas': 792}, {'Año': 2009, 'Horas': 848}, {'Año': 2010, 'Horas': 3982}, {'Año': 2011, 'Horas': 2953}, {'Año': 2012, 'Horas': 4368}, {'Año': 2013, 'Horas': 1545}, {'Año': 2014, 'Horas': 769}, {'Año': 2015, 'Horas': 930}, {'Año': 2016, 'Horas': 174}, {'Año': 2017, 'Horas': 38}]}


In [70]:
# Archivo para tercera, cuarta y quinta funcion.
"""Para esta funcion necesitamos:
item_id y sentiment_analysis: del archivo Australian_user_reviews_normalizada_SA
item_id title y release_year: del archivo output_steam_games_limpio"""

import pandas as pd

df_reviews = pd.read_csv('CSV/Australian_user_reviews_normalizada_SA.csv')
df_games = pd.read_csv('CSV/output_steam_games_limpio.csv')

columns_reviews = df_reviews[['item_id', 'user_id', 'sentiment_analysis', 'recommend']]
columns_games = df_games[['item_id', 'title', 'release_year']]

df_function_3_4_5 = pd.merge(columns_reviews, columns_games, on='item_id', how='inner')

# configuro todos los valores a 1 que seria lo negativo.
df_function_3_4_5['reviews_recommend'] = 1

# cuando cumpla condicion true + 1 o 2, se guarde un 2 que seria lo positivo.
condition = (df_function_3_4_5['recommend'] == True) & (df_function_3_4_5['sentiment_analysis'].isin([1, 2]))
df_function_3_4_5.loc[condition, 'reviews_recommend'] = 2

# Guardar el nuevo DataFrame en un archivo CSV
df_function_3_4_5.to_csv('Datasets/function_3_4_5.csv', index=False)

In [91]:
import pandas as pd
df_function_3_4_5 = pd.read_csv('Datasets/function_3_4_5.csv')
df_function_3_4_5.head()

,item_id,user_id,sentiment_analysis,recommend,title,release_year,reviews_recommend
0,1250,76561197970982479,1,True,Killing Floor,2009,2
1,1250,death-hunter,1,True,Killing Floor,2009,2
2,1250,DJKamBer,1,True,Killing Floor,2009,2
3,1250,diego9031,1,True,Killing Floor,2009,2
4,1250,76561198081962345,1,True,Killing Floor,2009,2


In [97]:
# Tercera funcion.
import pandas as pd

df_function_3 = pd.read_csv('Datasets/function_3_4_5.csv')

def UsersRecommend(year:int):
    df_filter = df_function_3[df_function_3['release_year'] == year]

    if df_filter.empty:
        return {"Mensaje": f"No se encontraron juegos recomendados para el año {year}"}

    # Filtrar "reviews_recommend" = 2 y contar por juego.
    recommended_games = df_filter[df_filter['reviews_recommend'] == 2].groupby('title').size().reset_index()
    
    # Ordenar los juegos.
    top_games = recommended_games.rename(columns={0: 'count'}).sort_values(by='count', ascending=False).head(3)

    # lista de diccionarios del top 3
    top_3 = [{"Puesto " + str(i + 1): game} for i, game in enumerate(top_games['title'])]
    return top_3
print(UsersRecommend(2012))

[{'Puesto 1': 'Counter-Strike: Global Offensive'}, {'Puesto 2': 'Borderlands 2'}, {'Puesto 3': 'Realm of the Mad God'}]


In [96]:
# Cuarta funcion.
import pandas as pd

df_function_4 = pd.read_csv('Datasets/function_3_4_5.csv')

def UsersNotRecommend(year:int):
    df_filter = df_function_4[df_function_4['release_year'] == year]

    if df_filter.empty:
        return {"Mensaje": f"No se encontraron juegos recomendados para el año {year}"}

    # Filtrar las filas donde "reviews_recommend" es igual a 1 y contar por juego
    not_recommended_games = df_filter[df_filter['reviews_recommend'] == 1].groupby('title').size().reset_index()

    # Ordenar los juegos por la cantidad de recomendaciones en orden descendente
    top_games = not_recommended_games.rename(columns={0: 'count'}).sort_values(by='count', ascending=False).head(3)

    # Crear una lista de diccionarios que representen el top 3
    top_3 = [{"Puesto " + str(i + 1): game} for i, game in enumerate(top_games['title'])]
    return top_3
print(UsersNotRecommend(2012))

[{'Puesto 1': 'Counter-Strike: Global Offensive'}, {'Puesto 2': 'Realm of the Mad God'}, {'Puesto 3': 'PlanetSide 2'}]


In [86]:
# Quinta funcion.

import pandas as pd

df_function_5 = pd.read_csv('Datasets/function_3_4_5.csv')

def sentiment_analysis(year:int):
    df_filter = df_function_5[df_function_5['release_year'] == year]

    if df_filter.empty:
        return {"Mensaje": f"No se encontraron registros para el año {year}"}

    # Remapear los valores de análisis de sentimiento a etiquetas.
    df_filter.loc[:, 'sentiment_analysis'] = df_filter['sentiment_analysis'].map({0: "Negative", 1: "Neutral", 2: "Positive"})

    # Agrupar y cuenta los grupos.
    analysis_count = df_filter['sentiment_analysis'].value_counts().to_dict()

    # Diccionario con las categorías y recuento.
    result = {key: value for key, value in analysis_count.items()}
    return result
print(sentiment_analysis(2012))

{'Neutral': 11966, 'Positive': 3081, 'Negative': 1221}
